In [1]:
import sys

sys.path.append("..")

In [13]:
import ast
import json

from typing import Tuple

import pandas as pd
import pytorch_lightning as pl
from datasets import Dataset, DatasetDict, Audio
from sklearn.model_selection import train_test_split
from huggingface_hub import HfApi

from src.constants import METADATA_CSV_PATH, AUDIO_DATA_PATH, REPO_ID

pl.seed_everything(42)

PUSH_TO_HUB = False  # Set to True to push datasets to the Hugging Face Hub
PUSH_TO_HUB = True

Seed set to 42


## Load dataset

- Load metadata 
- Clean and preprocess columns
- Push just captions version
- Create 3 versions of audio dataset, 10% subset, 25% subset and 100% subset

In [3]:
metadata_df = pd.read_csv(METADATA_CSV_PATH)
metadata_df

,id,aspect_list,prediction,filename
0,fc382d0ae2b46bd0538115871469af28,"['ambient noises', 'ambient sounds', 'digital ...",This energetic electronic track is marked by i...,fc382d0ae2b46bd0538115871469af28.wav
1,2bb4b92eba27ea6acac7473a6d6b24d0,"['cheerful', 'country music', 'fast tempo', 'n...",This upbeat country-rock tune kicks off with a...,2bb4b92eba27ea6acac7473a6d6b24d0.wav
2,0f698b0297962863c59b64ab2382760c,"['acoustic drums', 'acoustic guitar', 'aggress...",This epic heavy metal track begins with an omi...,0f698b0297962863c59b64ab2382760c.wav
3,5456c0ff1e48d414af58de19a21e2075,"['acoustic drums', 'acoustic guitar', 'bass gu...",This upbeat pop-rock track features a driving ...,5456c0ff1e48d414af58de19a21e2075.wav
4,bf27536a1f56feefc2b606c4305dbe21,"['male voice', 'movie music', 'passionate']",A sweeping orchestral score unfolds over a bac...,bf27536a1f56feefc2b606c4305dbe21.wav
...,...,...,...,...
5353,938aaac2ca856fb6b0cd268bb5633322,"['emotional', 'intense', 'medium tempo', 'mell...","This mid-tempo track unfolds at a steady pace,...",938aaac2ca856fb6b0cd268bb5633322.wav
5354,63dacd2d7e887f90d49dc7614b0dd3b8,"['chaotic', 'danceable', 'energetic', 'piano']",This high-energy track bursts forth from a fra...,63dacd2d7e887f90d49dc7614b0dd3b8.wav
5355,540b04f153f687abb8e6d4d0fd242b2d,"['energetic', 'fast tempo', 'piano', 'upbeat']",This high-energy track unfolds at a rapid-fire...,540b04f153f687abb8e6d4d0fd242b2d.wav
5356,484728dc21e8a87296df0f71db0be3a6,"['acoustic guitar', 'claps', 'dance', 'energet...",This upbeat track kicks off with bouncy acoust...,484728dc21e8a87296df0f71db0be3a6.wav


In [4]:
# Load concept-to-tags mapping
CONCEPTS = json.load(open("../data/concepts_to_tags.json", "r"))

print("Available concept categories:")
for cat, tags in CONCEPTS.items():
    print(f"  {cat}: {len(tags)} tags (e.g., {tags[:3]})")

Available concept categories:
  tempo: 50 tags (e.g., ['medium tempo', 'slow tempo', 'fast tempo'])
  genre: 50 tags (e.g., ['rock', 'pop', 'electronic music'])
  mood: 50 tags (e.g., ['emotional', 'passionate', 'energetic'])
  instrument: 50 tags (e.g., ['acoustic drums', 'electric guitar', 'bass guitar'])


In [5]:
# Create reverse mapping
TAG_TO_CATEGORY = {}
for cat, tags in CONCEPTS.items():
    for tag in tags:
        TAG_TO_CATEGORY[tag] = cat

In [6]:
# Categorize tags in aspect_list column
def categorize_tags(aspect_list):
    tags = ast.literal_eval(aspect_list)
    aspect_list_categorized = {}
    for tag in tags:
        category = TAG_TO_CATEGORY.get(tag)
        category_columns = f"{category}_aspects"
        if category_columns not in aspect_list_categorized:
            aspect_list_categorized[category_columns] = []
        aspect_list_categorized[category_columns].append(tag)
    for category in CONCEPTS.keys():
        category_columns = f"{category}_aspects"
        if category_columns not in aspect_list_categorized:
            aspect_list_categorized[category_columns] = []
    return aspect_list_categorized

In [7]:
metadata_df = pd.concat(
    [
        metadata_df,
        metadata_df["aspect_list"].apply(categorize_tags).apply(pd.Series),
    ],
    axis=1,
)
metadata_df.rename(columns={"prediction": "caption"}, inplace=True)
metadata_df.head()

,id,aspect_list,caption,filename,genre_aspects,instrument_aspects,mood_aspects,tempo_aspects
0,fc382d0ae2b46bd0538115871469af28,"['ambient noises', 'ambient sounds', 'digital ...",This energetic electronic track is marked by i...,fc382d0ae2b46bd0538115871469af28.wav,"[ambient noises, ambient sounds, electronic mu...","[digital drums, groovy bass line, keyboard, ke...","[emotional, energetic]",[medium tempo]
1,2bb4b92eba27ea6acac7473a6d6b24d0,"['cheerful', 'country music', 'fast tempo', 'n...",This upbeat country-rock tune kicks off with a...,2bb4b92eba27ea6acac7473a6d6b24d0.wav,"[country music, rock]","[no voice, trumpets]","[cheerful, playful]","[fast tempo, upbeat]"
2,0f698b0297962863c59b64ab2382760c,"['acoustic drums', 'acoustic guitar', 'aggress...",This epic heavy metal track begins with an omi...,0f698b0297962863c59b64ab2382760c.wav,"[movie music, rock music]","[acoustic drums, acoustic guitar, bass, bass g...","[aggressive, gentle, heavy metal, intense]",[medium tempo]
3,5456c0ff1e48d414af58de19a21e2075,"['acoustic drums', 'acoustic guitar', 'bass gu...",This upbeat pop-rock track features a driving ...,5456c0ff1e48d414af58de19a21e2075.wav,"[country music, pop, pop rock, rock]","[acoustic drums, acoustic guitar, bass guitar,...","[cheerful, energetic, fun, happy]","[fast tempo, groovy, medium tempo, upbeat]"
4,bf27536a1f56feefc2b606c4305dbe21,"['male voice', 'movie music', 'passionate']",A sweeping orchestral score unfolds over a bac...,bf27536a1f56feefc2b606c4305dbe21.wav,[movie music],[male voice],[passionate],[]


In [8]:
def create_balanced_splits(
    df: pd.DataFrame,
    train_size: float = 0.7,
    valid_size: float = 0.15,
    test_size: float = 0.15,
    random_state: int = 42
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    train_df, temp_df = train_test_split(
        df,
        train_size=train_size,
        random_state=random_state
    )
    
    valid_ratio = valid_size / (valid_size + test_size)
    valid_df, test_df = train_test_split(
        temp_df,
        train_size=valid_ratio,
        random_state=random_state
    )
    
    return train_df, valid_df, test_df


def create_percentage_subset(
    train_df: pd.DataFrame,
    percentage: float,
    random_state: int = 42
) -> pd.DataFrame:
    subset_df, _ = train_test_split(
        train_df,
        train_size=percentage,
        random_state=random_state
    )
    return subset_df

def prepare_dataset_dict_with_audio(
    train_df: pd.DataFrame,
    valid_df: pd.DataFrame,
    test_df: pd.DataFrame,
    include_audio: bool = False,
    subset_name: str = "100"
) -> DatasetDict:
    """Prepare dataset with optional audio column."""
    columns = [
        "id", "caption", "aspect_list",
        "genre_aspects", "mood_aspects", 
        "instrument_aspects", "tempo_aspects"
    ]
    
    if include_audio:
        columns.append("file_name")
    
    train_clean = train_df[columns].copy()
    valid_clean = valid_df[columns].copy()
    test_clean = test_df[columns].copy()
    
    dataset_dict = DatasetDict({
        "train": Dataset.from_pandas(train_clean, preserve_index=False),
        "validation": Dataset.from_pandas(valid_clean, preserve_index=False),
        "test": Dataset.from_pandas(test_clean, preserve_index=False),
    })
    
    print(f"\nDataset config '{subset_name}' created:")
    print(f"  Train: {len(dataset_dict['train'])} samples")
    print(f"  Valid: {len(dataset_dict['validation'])} samples")
    print(f"  Test:  {len(dataset_dict['test'])} samples")
    print(f"  Audio: {include_audio}")
    
    return dataset_dict

## Create balanced subsets based on aspect_list

In [9]:
metadata_no_audio = metadata_df.drop(columns=['file_name'] if 'file_name' in metadata_df.columns else [])

train_df_no_audio, valid_df_no_audio, test_df_no_audio = create_balanced_splits(metadata_no_audio)

train_10_no_audio = create_percentage_subset(train_df_no_audio, percentage=0.10)
valid_10_no_audio = create_percentage_subset(valid_df_no_audio, percentage=0.10)
test_10_no_audio = create_percentage_subset(test_df_no_audio, percentage=0.10)

train_25_no_audio = create_percentage_subset(train_df_no_audio, percentage=0.25)
valid_25_no_audio = create_percentage_subset(valid_df_no_audio, percentage=0.25)
test_25_no_audio = create_percentage_subset(test_df_no_audio, percentage=0.25)

In [10]:
metadata_df['file_name'] = metadata_df.apply(
    lambda row: str(AUDIO_DATA_PATH / row['filename']) if 'filename' in row else None, 
    axis=1
)

train_df_audio, valid_df_audio, test_df_audio = create_balanced_splits(metadata_df)

train_10_audio = create_percentage_subset(train_df_audio, percentage=0.10)
valid_10_audio = create_percentage_subset(valid_df_audio, percentage=0.10)
test_10_audio = create_percentage_subset(test_df_audio, percentage=0.10)

train_25_audio = create_percentage_subset(train_df_audio, percentage=0.25)
valid_25_audio = create_percentage_subset(valid_df_audio, percentage=0.25)
test_25_audio = create_percentage_subset(test_df_audio, percentage=0.25)

In [11]:
dataset_full = prepare_dataset_dict_with_audio(
    train_df_no_audio, valid_df_no_audio, test_df_no_audio, 
    include_audio=False, subset_name="default"
)
dataset_25pct = prepare_dataset_dict_with_audio(
    train_25_no_audio, valid_25_no_audio, test_25_no_audio,
    include_audio=False, subset_name="25pct"
)
dataset_10pct = prepare_dataset_dict_with_audio(
    train_10_no_audio, valid_10_no_audio, test_10_no_audio,
    include_audio=False, subset_name="10pct"
)
dataset_full_audio = prepare_dataset_dict_with_audio(
    train_df_audio, valid_df_audio, test_df_audio,
    include_audio=True, subset_name="audio"
)
dataset_full_audio = dataset_full_audio.cast_column("file_name", Audio())
dataset_25pct_audio = prepare_dataset_dict_with_audio(
    train_25_audio, valid_25_audio, test_25_audio,
    include_audio=True, subset_name="25pct-audio"
)
dataset_25pct_audio = dataset_25pct_audio.cast_column("file_name", Audio())
dataset_10pct_audio = prepare_dataset_dict_with_audio(
    train_10_audio, valid_10_audio, test_10_audio,
    include_audio=True, subset_name="10pct-audio"
)
dataset_10pct_audio = dataset_10pct_audio.cast_column("file_name", Audio())


Dataset config 'default' created:
  Train: 3750 samples
  Valid: 804 samples
  Test:  804 samples
  Audio: False

Dataset config '25pct' created:
  Train: 937 samples
  Valid: 201 samples
  Test:  201 samples
  Audio: False

Dataset config '10pct' created:
  Train: 375 samples
  Valid: 80 samples
  Test:  80 samples
  Audio: False

Dataset config 'audio' created:
  Train: 3750 samples
  Valid: 804 samples
  Test:  804 samples
  Audio: True

Dataset config '25pct-audio' created:
  Train: 937 samples
  Valid: 201 samples
  Test:  201 samples
  Audio: True

Dataset config '10pct-audio' created:
  Train: 375 samples
  Valid: 80 samples
  Test:  80 samples
  Audio: True


In [ ]:
if PUSH_TO_HUB:
    api = HfApi()
    api.upload_file(
        path_or_fileobj="../docs/DATASET_CARD.md",
        path_in_repo="README.md",
        repo_id=REPO_ID,
        repo_type="dataset",
    )

CommitInfo(commit_url='https://huggingface.co/datasets/bsienkiewicz/ConceptCaps/commit/9b0900ade0f48ce9d8683b589d3d150ccc9d2650', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='9b0900ade0f48ce9d8683b589d3d150ccc9d2650', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/bsienkiewicz/ConceptCaps', endpoint='https://huggingface.co', repo_type='dataset', repo_id='bsienkiewicz/ConceptCaps'), pr_revision=None, pr_num=None)

In [ ]:
if PUSH_TO_HUB:
    print(f"\nPushing all configurations to {REPO_ID}...")
    
    print("Pushing 'default' configuration...")
    dataset_full.push_to_hub(REPO_ID, config_name="default", private=False)
    
    print("Pushing '25pct' configuration...")
    dataset_25pct.push_to_hub(REPO_ID, config_name="25pct", private=False)
    
    print("Pushing '10pct' configuration...")
    dataset_10pct.push_to_hub(REPO_ID, config_name="10pct", private=False)
    
    print("Pushing 'audio' configuration (full with audio)...")
    dataset_full_audio.push_to_hub(REPO_ID, config_name="audio", private=False)
    
    print("Pushing '25pct-audio' configuration...")
    dataset_25pct_audio.push_to_hub(REPO_ID, config_name="25pct-audio", private=False)
    
    print("Pushing '10pct-audio' configuration...")
    dataset_10pct_audio.push_to_hub(REPO_ID, config_name="10pct-audio", private=False)


Pushing all configurations to bsienkiewicz/ConceptCaps...
Pushing 'default' configuration...


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

/home/bruno/miniconda3/envs/concept-caps/lib/python3.12/site-packages/huggingface_hub/hf_api.py:9662: UserWarning: Warnings while validating metadata in README.md:
- The task_categories "audio-to-text" is not in the official list: text-classification, token-classification, table-question-answering, question-answering, zero-shot-classification, translation, summarization, feature-extraction, text-generation, fill-mask, sentence-similarity, text-to-speech, text-to-audio, automatic-speech-recognition, audio-to-audio, audio-classification, audio-text-to-text, voice-activity-detection, depth-estimation, image-classification, object-detection, image-segmentation, text-to-image, image-to-text, image-to-image, image-to-video, unconditional-image-generation, video-classification, reinforcement-learning, robotics, tabular-classification, tabular-regression, tabular-to-text, table-to-text, multiple-choice, text-ranking, text-retrieval, time-series-forecasting, text-to-video, image-text-to-text, i